# Actividad M1 - Aspiradora
**Rafael Alfonso Gómez González**

In [363]:
import agentpy as ap
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
from random import randrange

In [365]:
class Vacuum(ap.Model):

    def setup(self):
        
        # Create agents (robot aspirador) using Number of robots parameter
        n_robots = int(self.p['Number of robots'])
        n_dirty = int(self.p['Dirtiness percentage'] * (self.p.size**2)) #Returns Int
        
        robots = self.robotAgent = ap.AgentList(self, n_robots)
        spaces = self.spacesAgent = ap.AgentList(self, self.p.size**2)
        
        # Condition:
        #   0:Clean
        #   1:Dirty
        #   2:Robots
        spaces.condition = 0
        dirtyness = spaces.random(n_dirty)
        dirtyness.condition = 1
      
        robots.condition = 2
        coordinate = (1,1)
        coordiantes = [coordinate] * n_robots

        # Create grid (room)
        self.room = ap.Grid(self, [self.p.size]*2, track_empty=True)
        self.room.add_agents(spaces, random=True, empty=True)
        self.room.add_agents(robots, positions=coordiantes, empty= True)
        
    def step(self):
        robotsPositions = self.robotAgent.select(self.robotAgent.condition == 2)
        # Select dirty spaces
        for robot in robotsPositions:
            for neighbor in self.room.neighbors(robot):
                if neighbor.condition == 1:
                    neighbor.condition = 0
                if neighbor.condition == 2:
                    self.room.move_by(robot, (0,0))
            self.room.move_by(robot,(random.randrange(-1,2),random.randrange(-1,2)))
        
        dirtyness = self.spacesAgent.select(self.spacesAgent.condition == 1)
        if len(dirtyness) == 0:
            self.stop()     

    def end(self):
        # Document a measure at the end of the simulation
        cleaned_spaces = len(self.spacesAgent.select(self.spacesAgent.condition == 0))
        self.report('Percentage of cleaned spaces',
                    cleaned_spaces / len(self.spacesAgent))


In [366]:
# Define parameters
parameters = {
    'Number of robots': 200,
    'Dirtiness percentage': .7, # Percentage of grid covered by dirt
    'size': 40, # Height and length of the room
    'steps': 300,
}

In [367]:
def animation_plot(model, ax):
    attr_grid = model.room.attr_grid('condition')
    color_dict = {0:'#092cad', 1:'#d62c2c', 2:'#a4ba29',3:"#e5e5e5" , None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a roomba\n"
                 f"Time-step: {model.t}, Dirty left: "
                 f"{len(model.spacesAgent.select(model.spacesAgent.condition == 1))}"
                 )

fig, ax = plt.subplots()
model = Vacuum(parameters)
#model.run().save()
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=60))